In [1]:
!pip install faiss-cpu pypdf2

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/14.6 MB 1.5 MB/s eta 0:00:10
    --------------------------------------- 0.3/14.6 MB 1.9 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.6 MB 2.5 MB/s eta 0:00:06
   - -------------------------------------- 0.6/14.6 MB 2.3 MB/s eta 0:00:06
   --- ------------------------------------ 1.1/14.6 MB 3.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.2/14.6 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.8/14.6 MB 4.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.6 MB 5.0 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.6 MB 5.9 MB/s eta 0:00:02
   --------- ------------------------------ 3.5/14.6 MB 5.8 MB/s eta 0:00:02
   ---------- ----------------------------- 4.0/14.6 MB 5.9 MB/s eta 0:00:02
   ----------

In [39]:
import faiss
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import fitz
import itertools

In [46]:
pdf_file_path = 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf'
# with fitz.open(pdf_file_path) as doc:
#         page_contents = [page.get_text() for page in doc]
pdf_loader = PyPDFLoader("C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf")
raw_documents= pdf_loader.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
documents = text_splitter.split_documents(raw_documents)

In [47]:
embedding_model = OllamaEmbeddings(model='nomic-embed-text',show_progress=True)

In [48]:
batch_size = 50
texts=[doc.page_content for doc in documents]
#document_texts = [doc.page_content for doc in documents]
documents_embeddings = []

In [49]:
for i in range(0, len(texts),batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embedding_model.embed_documents(batch_texts)
    documents_embeddings.extend(batch_embeddings)


OllamaEmbeddings: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:45<00:00,  1.10it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:43<00:00,  1.16it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:43<00:00,  1.15it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:43<00:00,  1.14it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:39<00:00,  1.27it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:44<00:00,  1.13it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:43<00:00,  1.15it/s]

OllamaEmbeddings: 100%|██████████| 30/30 [00:28<00:00,  1.07it/s]


In [50]:
import numpy as np

In [51]:
#create FAISS index and add the embeddings
document_embeddings_np = np.array(documents_embeddings)
dimension = document_embeddings_np.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings_np)

In [52]:
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document

In [53]:
docstore = InMemoryDocstore(dict(enumerate(documents)))
index_to_docstore_id = {i: i for i in range(len(documents))}

In [54]:
vector_store = FAISS(embedding_model, faiss_index, docstore, index_to_docstore_id)

In [55]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [56]:
# LLM from Ollama
local_model = "knoopx/hermes-2-pro-mistral:7b-q8_0"
llm = ChatOllama(model=local_model)

In [57]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task to phrase the user question in the best possible way in order to get the most accurate answer from the document loaded. Your response should also separately contain the precise data which was referred to in the document in order to come up with your response to the question. Make sure to refer to the exact point in the document in the form of page numbers, etc.
    Original question: {question}""",
)

In [58]:
retriever = MultiQueryRetriever.from_llm(
    vector_store.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
Ensure that you mention the page number or position in the document from which you arrived at your response. 
Cite the original wordings in the document along with it. Make this a separate line in the response along with your answer.
Try to make your responses as concise as possible.

"""

prompt = ChatPromptTemplate.from_template(template)

In [59]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [60]:
chain.invoke("What access control procedures are in place?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:05<00:00,  5.33s/it]


"Access control procedures mentioned in the document include:\n\n1. On page 60-62 (GCP SOC 2 doc): Customers are responsible for provisioning, maintaining, monitoring, and disabling end users' access according to their internal access management policies. The entity implements logical access security software, infrastructure, and architectures over protected information assets to protect them from security events that meet the entity's objectives. Prior to issuing system credentials and granting system access, the entity registers and authorizes new internal and external users whose access is administered by the entity. For those users whose access is administered by the entity, user system credentials are removed when access is no longer authorized.\n\n2. On page 67 (GCP SOC 2 doc): The entity authorizes, modifies, or removes access to data, software, functions, and other protected information assets based on roles. This is done within the customer's environment consistent with custom

In [61]:
chain.invoke("When was the access control policy last reviewed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.75s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  9.08it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]


'The access control policy last review information is not available in the provided excerpts. Please refer to page 144 of the document "GCP-[FALL-2023] GCP SOC 2..pdf" for the results of the tests performed by EY on critical access group reviews, which include the periodicity of the reviews but does not explicitly mention when the policy was last reviewed. The original wording from the document reads: "Inspected relevant documentation and determined that critical access group reviews were done on a periodic basis and scoping was determined accordingly." (Page 144)\n\nAnswer: The information regarding the last review of the access control policy is not available in the given excerpts. Please refer to page 144 for details on the critical access group reviews, but note that it does not explicitly mention the last review date of the access control policy itself.'

In [62]:
chain.invoke("What is the password management policy in place?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 12.81it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.04it/s]


"The password management policy in place includes the following:\n\n1. Customers are responsible for reviewing users' access rights periodically, consistent with customer organizational policies, to mitigate the risk of inappropriate access (Page 63).\n2. The entity implements logical access security software, infrastructure, and architectures over protected information assets to protect them from security events to meet the entity's objectives (Page 63).\n3. Two-step verification is implemented on privileged administrator accounts (Page 64).\n4. Prior to issuing system credentials and granting system access, the entity registers and authorizes new internal and external users whose access is administered by the entity. For those users whose access is administered by the entity, user system credentials are removed when user access is no longer authorized (Page 67).\n\nCommon Criteria 6.1 and 6.3 are relevant to these policies (Page 63, 64, and 67)."

In [63]:
chain.invoke("What procedures are followed for decommissioning of equipments?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.31s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  9.47it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


'When a hard drive is retired, authorized individuals verify that the disk is erased by writing zeros to the drive and performing a multiple-step verification process to help ensure the drive contains no data. If the drive cannot be erased for any reason, it is stored securely until it can be physically (Page 53).\n\nProcedures followed for decommissioning of equipments:\n1. Hard drives are synchronized with a standard base image and utilize technologies such as Full Disk Encryption (FDE) and drive locking to protect data at rest.\n2. When a component fails a performance test at any point during its lifecycle, it is removed from inventory and retired.\n3. Google utilizes barcodes and asset tags to track the status and location of data center equipment from acquisition to installation, retirement, and destruction.\n4. If PII on removable media leaving Google facilities is not approved, it is encrypted.\n5. When a hard drive is retired, it undergoes a process where authorized individuals

In [64]:
chain.invoke("What steps are taken to ensure removal of client information prior to decommissioning of equipments?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:07<00:00,  7.32s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.24it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


'When a hard drive is retired, authorized individuals verify that the disk is erased by writing zeros to the drive and performing a multiple-step verification process to help ensure the drive contains no data. If the drive cannot be erased for any reason, it is stored securely until it can be physically destroyed. This process is mentioned on page 53 of the document “GCP [FALL-2023] GCP SOC 2..pdf”.\n\nReference: https://i.imgur.com/1Z7XzNr.png (Page 53)'

In [65]:
chain.invoke("When were information security policies and procedures updated?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.24s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.83it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]


'The information security policies and procedures were updated on \'1 October 20XX\' (year not clearly mentioned). This information can be found on page 35 of the document, which states: "Information Security Policies and Procedures were updated on 1 October 20XX."\n\nOriginal wording from the document: "Information Security Policies and Procedures were updated on 1 October 20XX."'

In [66]:
chain.invoke("How are emergency changes performed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:11<00:00, 11.35s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 15.54it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.36it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.87it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.09it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.32it/s]


'Emergency changes are performed through the Change Management process. According to page 50 of the document, "Emergency Changes: Any change that must be made immediately to avoid a significant impact to service or customer data is considered an emergency change." The process requires documentation and review prior to implementation, as well as post-implementation review and communication to the appropriate stakeholders. It also states that such changes are tracked in the same way as regular changes, with necessary escalation mechanisms in place for urgent issues. \n\nOriginal wording from the document: "Emergency Changes: Any change that must be made immediately to avoid a significant impact to service or customer data is considered an emergency change. Emergency changes follow the same change management process, but are documented and reviewed prior to implementation, implemented, and post-implementation reviewed. The change is communicated to the appropriate stakeholders as per the 

In [67]:
chain.invoke("What cryptographic controls are in place to encrypt the data at rest and in motion?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.17s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.81it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.12it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.39it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 15.86it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.32it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.76it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 12.28it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]


"In order to find information about cryptographic controls for data encryption at rest and in motion, I would need access to the specific audit report or documentation related to Google Cloud's security measures. Please provide the relevant document or report so that I can locate the required information and respond accordingly. Make sure to mention the page number or position in the document along with the original wording from the source."

In [68]:
chain.invoke("When was the incident management policy last reviewed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 18.18it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.74it/s]


'I am unable to find information about the incident management policy review date in the provided document excerpts. Please provide the complete audit report or the specific section containing details about the incident management policy for further assistance.'

In [69]:
chain.invoke("When was the incident response procedure tested?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.21s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.42it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s]


'The incident response procedure was tested during the audit process, specifically during the review of the "Incident Alert and Recording" section on page 51 of the document. The text states, "Log sources are used to generate alerts whenever an anomaly occurs... An anomaly may also be manually documented by a Google employee when an issue is identified or in response to a customer service request." (GCP-[FALL-2023] GCP SOC 2..pdf, p. 51)'

In [70]:
chain.invoke("How are incidents classified?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.76s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.34it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 14.26it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  5.81it/s]


'Incidents are classified based on their criticality. The classification, prioritization, and escalation of security incidents are identified, and mechanisms are defined to measure and monitor the type and scope of security incidents. This is outlined on page 170 of the document: "were in place which outline a quick, effective, and orderly response to information security incidents. In addition, classification, prioritization, and escalation of security incidents per criticality are also identified and mechanisms are defined to measure and monitor the type and scope of security incidents." No deviations were noted in this area.\n\nReference(s):\n- Page 170: "were in place which outline a quick, effective, and orderly response to information security incidents. In addition, classification, prioritization, and escalation of security incidents per criticality are also identified and mechanisms are defined to measure and monitor the type and scope of security incidents."'

In [71]:
chain.invoke("What procedures are in place to perform root cause analysis for an incident?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:05<00:00,  5.46s/it]


"I am unable to find specific procedures for performing root cause analysis mentioned in the provided documents. The information given pertains mainly to Google Cloud services, their functionalities, and audit report findings. It does not explicitly detail the procedures for conducting a root cause analysis in case of an incident.\n\nSource:\nDocument(metadata={'source': 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf', 'page': 18}, page_content='dependency trees), and categorizing the content into a rich taxonomy. The API can be called by \\npassing the content directly or by referring to a document in Google Cloud Storage.')\n\nDocument(metadata={'source': 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf', 'page': 75}, page_content='requiring syst

In [72]:
chain.invoke("What physical perimeter controls are in place?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.08s/it]


"I could not find any information about physical perimeter controls in the provided document excerpts. The mentioned Google Cloud services and features are described at various page numbers, but none specifically address physical perimeter controls. Please provide more context or refer to a different section of the document if you're looking for information on this particular topic.\n\nPage numbers mentioned in the request: 18, 22, 73, and 103."

In [73]:
chain.invoke("What procedures are in place to deploy patches throughout the IT infrastructure?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.27s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 17.27it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 10.98it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.77it/s]


'The document does not specifically mention procedures for deploying patches throughout the IT infrastructure. However, it does discuss security updates and vulnerability management in the context of Google\'s Cloud Security Command Center. This service provides visibility into security and network traffic data across multi-cloud and on-premises environments (page 51). It also offers automated policy enforcement for identifying and responding to threats (page 52). These capabilities help ensure that patches and updates are applied promptly and consistently across the infrastructure, although the specific procedures for patch deployment are not detailed in the provided excerpts.\n\n(Source: "GCP-[FALL-2023] GCP SOC 2..pdf", pages 51-52)'

In [74]:
chain.invoke("How is a back up of the data performed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.54s/it]


'A backup of the data is performed as per the data retention and deletion policies. The process involves the use of data deletion tools that verify the backup data is deleted following the configured retention period, as part of the deletion mechanism process. This information can be found on page 77 of the document: "was disposed of as per the data retention and deletion policies. No deviations noted. Inspected a sample product and determined data deletion tools verified that backup data was deleted following the configured retention period, as part of the deletion mechanism process. No deviations noted."'

In [75]:
chain.invoke("What backgound checks are performed during hiring of an employee?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.85s/it]


"Background checks performed during hiring of an employee include criminal, credit, and/or security checks on all potential employees, temporary workers, and independent contractors, as well as verification of the individual's education, previous employment, and referrals (Source: Page 43, Document metadata {'source': 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf'}). The specifics or extent of background checks performed depend on the position and location for which the individual is applying."

In [76]:
chain.invoke("What is done to ensure that periodic penetration tests are performed for the infrastructure, devices, and end-points?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.60s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 12.04it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.06it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 13.63it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


'To ensure that periodic penetration tests are performed for the infrastructure, devices, and end-points, Google has the following measures in place:\n\n1. Penetration tests are performed at least annually. (CC4.1)\n2. The organization performs penetration tests by qualified internal personnel or an external service provider at least annually. (Inquired of the Program Manager)\n\nNo deviations were noted in these procedures during the testing process, as confirmed by the results shared by EY after their inspection. \n\nReference(s):\n- Page 54: "logs is restricted to authorized personnel. Security event logs are monitored continuously using a Google proprietary Security Event Management (SEM) system to detect intrusion attempts and other security related events."\n- Page 168: "Control Description SOC 2 Criteria, Controls, Tests and Results of Tests \\n...112. Penetration tests are performed at least annually. CC4.1..."'